# Part 3 -- Feature Selection

The libraries that we are going to use:

In [17]:
import pandas as pd 
import math
from math import log
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, accuracy_score, auc
from sklearn.model_selection import KFold
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import matplotlib.pyplot as plt1
import matplotlib.pyplot as plt2

from wordcloud import STOPWORDS
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import csv
import random
import math
import operator
from operator import itemgetter
from collections import Counter

We read our training and testing data:

In [18]:
Train_data = pd.read_csv(sep='\t',filepath_or_buffer='train.tsv')
Test_data = pd.read_csv(sep='\t',filepath_or_buffer='test.tsv')
target = Train_data['Label']

### Our Cross-Validation function

In [19]:
cross_val_instance = 0

def cross_validate(clf,train_data,target_data):
    global cross_val_instance    # Needed to modify global copy of a global variable
    
    kf = KFold(n_splits=10)
    average_accuracy =0
    fold = 0
    for train_index, test_index in kf.split(train_data):
        cross_val_instance += 1
      
        test = train_data.loc[test_index, train_data.columns]
        train = train_data.loc[train_index, train_data.columns]
        target = target_data[train_index]
        clf_cv = clf.fit(train, target)
        yPred = clf_cv.predict(test)
        fold += 1
        print ("Fold " + str(fold)+"\n\n")
        target = target_data[test_index]
        accuracy = accuracy_score(target, yPred)
        # mylist['Accuracy'].append(accuracy)
        print("Accuracy: ", accuracy)
        average_accuracy+= accuracy
    average_accuracy = average_accuracy/10
    print("Average accuracy = ",average_accuracy)
    return average_accuracy
        

In [20]:
Train_data.head()

,Attribute1,Attribute2,Attribute3,Attribute4,Attribute5,Attribute6,Attribute7,Attribute8,Attribute9,Attribute10,...,Attribute13,Attribute14,Attribute15,Attribute16,Attribute17,Attribute18,Attribute19,Attribute20,Label,Id
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,67,A143,A152,2,A173,1,A192,A201,1,10101
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,22,A143,A152,1,A173,1,A191,A201,2,10102
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,49,A143,A152,1,A172,2,A191,A201,1,10103
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,45,A143,A153,1,A173,2,A191,A201,1,10104
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,53,A143,A153,2,A173,2,A191,A201,2,10105


## Data preprocessing

In [21]:
categories = ["Attribute1","Attribute3","Attribute4","Attribute6","Attribute7","Attribute9","Attribute10","Attribute10","Attribute12","Attribute14","Attribute15", "Attribute17", "Attribute19","Attribute20"]

proccessedData_train = Train_data.copy()
proccessedData_test = Test_data.copy()
for x in categories:
    converted = pd.Categorical(Train_data[x])
    proccessedData_train[x] = converted.codes
    converted_test = pd.Categorical(Test_data[x])
    proccessedData_test[x] = converted_test.codes
    
print(proccessedData_train)

     Attribute1  Attribute2  Attribute3  Attribute4  Attribute5  Attribute6  \
0             0           6           4           4        1169           4   
1             1          48           2           4        5951           0   
2             3          12           4           7        2096           0   
3             0          42           2           3        7882           0   
4             0          24           3           0        4870           0   
5             3          36           2           7        9055           4   
6             3          24           2           3        2835           2   
7             1          36           2           1        6948           0   
8             3          12           2           4        3059           3   
9             1          30           4           0        5234           0   
10            1          12           2           0        1295           0   
11            0          48           2           9 

### Our gain calculation

In [22]:
def get_number_of_Good(data):
    return  len(data[data["Label"]==1])
def get_number_of_Bad(data):
    return  len(data[data["Label"]==2])

### Our entropy calculation function

In [23]:
def entropy(data):
    
    if(data.shape[0]==0): #se periptwsh pou to dataset einai adeio
        return 0

    good_percent = get_number_of_Good(data) /data.shape[0]
    bad_percent = get_number_of_Bad(data)/data.shape[0]
    good_entropy =0.0
    if(get_number_of_Good(data)>0):
        good_entropy = -(good_percent*log(good_percent, 2))
    bad_entropy =0.0
    if(get_number_of_Bad(data)>0):
        bad_entropy = -(bad_percent*log(bad_percent, 2))
    entropy = good_entropy+bad_entropy
    return entropy

In [24]:
print(entropy(Train_data)) #entropy for the hole dataset

ValueError: math domain error

In [ ]:
def get_attribute_values(data):#synarthsh pou epistrefei ta values enos attribute
    values = []
    for x in data:
        #print(x)
        if(x not in values ):
            values.append(x)
    return values

In [ ]:

features = list(Train_data.columns.values)
features.remove('Id')
features.remove('Label')

#kanw ta attribute ayta categorical ['Attribute2', 'Attribute5', 'Attribute13']
#den kanoume kai ta alla (numerical)attribute dioti exoun to poly 4 diaforetikes times kai emeis theloume na exoume 5 kathgories

Train_data['Attribute2']= pd.qcut(Train_data['Attribute2'], 5, labels=["Attribute2_A","Attribute2_B","Attribute2_C","Attribute2_D","Attribute2_E"])
Train_data['Attribute5']= pd.qcut(Train_data['Attribute5'], 5, labels=["Attribute5_A","Attribute5_B","Attribute5_C","Attribute5_D","Attribute5_E"])
Train_data['Attribute13']= pd.qcut(Train_data['Attribute13'], 5, labels=["Attribute13_A","Attribute13_B","Attribute13_C","Attribute13_D","Attribute13_E"])

InformationGain_list = []
for Attribute in features:
    Attribute_values = get_attribute_values(Train_data[Attribute])
    attr_entropy = 0.0
    for value in Attribute_values:#for every value in Attribute values
        value_set= Train_data[Train_data[Attribute]==value]
        attr_entropy += (len(value_set)/(len(Train_data)))*entropy(value_set)  #ypologizoume to athrisma tou percetage(tou value) * entropy (tou value)
    attr_information_gain = entropy(Train_data)-attr_entropy
    InformationGain_list.append((Attribute,attr_information_gain))
#print("Infomation Gain List(of all attributes ):\n\n",InformationGain_list)
InformationGain_list

In [ ]:
print(sorted(InformationGain_list, key=lambda tup: tup[1]) )
InformationGain_list = sorted(InformationGain_list, key=lambda tup: tup[1])
InformationGain_list

In [ ]:
RANDOM_STATE = 123

rndf = RandomForestClassifier(warm_start=False, oob_score=False, max_features="sqrt", random_state=RANDOM_STATE)

In [ ]:
count = len(InformationGain_list)
average_accuracy_list=[]
average_accuracy_list_tuples = []
average_accuracy_list_accuracy =[]
average_accuracy_list_number_attributes = []
my_df = proccessedData_train
exclude = ['Id','Label']

for attr in InformationGain_list:#gia kathe stoixeio sthn InformationGain_list afairw panta to prwto (dhladh ayto me to mikrotero information_gain afou thn exw sortarei)
    print("count == ",count)
    
    excl = my_df.columns.difference(exclude)
    
    new_df_to_use  = my_df[excl]
    #efarmozoume cross validation gia kathe attribute pou kanoume exclude
    average_accuracy = cross_validate(rndf,new_df_to_use,proccessedData_train['Label'])
    average_accuracy_list.append((average_accuracy,count) )#to meiwn 2 gt sthn lista exclude einai hdh ta 'Id' 'Label    
    average_accuracy_list_accuracy.append(average_accuracy)
    average_accuracy_list_number_attributes.append(count)
    exclude.append(str(attr[0]))

    count -= 1
    
    if(count == -1):
        break;

In [ ]:
print(average_accuracy_list)

In [ ]:
print(average_accuracy_list_number_attributes)

In [ ]:
print(average_accuracy_list_accuracy)

In [ ]:
plt1.title('Accuracy Plot')

plt1.plot(average_accuracy_list_number_attributes, average_accuracy_list_accuracy)
plt1.ylabel('Accuracy')
plt1.xlabel('Number of Attributes')
plt1.show()

## The matrix of the features that we chose to remove in each repetition

In [ ]:
for x in InformationGain_list:
    print(x[0],"\t",x[1])

# We find the max accuracy 

In [ ]:
max_acc= average_accuracy_list[0][0] #h megisth accuracy
max_features = average_accuracy_list[0][1] #metablhth pou kratame to posa features htan mesa
for x in average_accuracy_list:
    if(x[0] >max_acc ):
        max_features= x[1] 
        max_acc = x[0]
print(max_acc,max_features)

### The testSet_Predictions.csv implementation

We base our implementation on the best classifier and the best number of features.

In [ ]:
count = len(InformationGain_list)
my_df = proccessedData_train
exclude = ['Id','Label']
for attr in InformationGain_list:
    print("count == ",count)
    exclude.append(str(attr[0]))
    count-=1
    if(count == max_features):
        print(excl)
        break;
 
excl = my_df.columns.difference(exclude)
print(excl)
new_train_data  = my_df[excl] #tous afairw ta attributes pou xreiazetai
new_test_data = my_df[excl]

## Random Forest (RF) Classification

In [ ]:
RANDOM_STATE = 123
rndf= RandomForestClassifier() 
clf_cv = rndf.fit(new_train_data, target)
predicted = clf_cv.predict(new_test_data)

In [ ]:
print(predicted)

In [ ]:
testSet = {'Client_ID':[],'Predicted_Label':[]}
label_dict = {'1':"Good" , '2':"Bad"}
counter = 1 
for x in predicted:
    testSet['Client_ID'].append(counter)
    counter+=1
    testSet['Predicted_Label'].append(str(label_dict[str(x)]))

In [ ]:
testSetpd = pd.DataFrame(data=testSet)
testSetcsv=testSetpd.ix[::, ['Client_ID', 'Predicted_Label']]

testSetpd

In [ ]:
testSetcsv.to_csv(path_or_buf='testSet_Predictions.csv', sep = '\t')